<a href="https://colab.research.google.com/github/katrin2202/text-summarization-in-nlp/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [7]:
#новости
from datasets import load_dataset
dataset = load_dataset("mlsum", "ru")

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
for i in range(5):
  print(f"{i}. TEXT")
  print(dataset['train'][i]['text'])
  print(f"summary: {dataset['train'][i]['summary']}")
  print(f"title: {dataset['train'][i]['title']}")

0. TEXT
Сладострастник в течение трех лет преследовал подростка в надежде совратить его. Как сообщили “МК” в следственном отделе по Хорошевскому району СУ СК при Прокуратуре РФ по Москве, 26 августа 2006 года 13-летний Павел вместе с другом отдыхал на берегу Москвы–реки рядом с Крылатским мостом. Там к ребятам подошел мужчина. Новый знакомый представился Евгением и предложил вместе пообедать в ресторане быстрого питания, а потом искупаться. Именно там, на берегу, педагог начал приставать к мальчику. Школьник убежал, но педофил успел снять голого подростка на мобильный телефон. После этого жизнь мальчика превратилась в сущий ад. Евгений узнал, где живет Павел, и стал шантажировать его. Этот кошмар продолжался три года. Преподаватель угрожал показать фотографию друзьям и знакомым Павла. Негодяй исписал непотребными надписями стены подъезда, где проживали друзья школьника. В один из дней он приехал в Сергиев Посад, к бабушке мальчика, и там накинулся на школьника с ножом. Наконец, отчаявш

In [1]:
%%capture
# Установка библиотек
!pip install datasets
!apt install git-lfs
!pip install transformers
!pip install sentencepiece 
!pip install rouge_score

In [12]:
# Импорт библиотек
import numpy as np
# import tensorflow
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
# import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric

In [71]:
# загрузка параметров
raw_datasets = load_dataset("mlsum", "ru")
metric = load_metric("rouge")
nltk.download('punkt')

  0%|          | 0/3 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
# Ввести свой ключ huggingface hyb
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [72]:
# Определение параметров
REPO = "UrukHan/t5-russian-summarization"  # Введите наазвание название репозитория
MODEL_NAME = "Katerina-gopher/t5-russian-summarization" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8 
DATASET = 'mlsum'   # Введите наазвание название датасета

In [74]:
# Загрузка датасета
data = load_dataset(DATASET, "ru")

  0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [58]:
model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности
# Закоментить после первого соъранения в репозиторий свой необъязательно
# tokenizer.push_to_hub("Katerina-gopher/t5-russian-summarization", create_pr=1) 
# tokenizer.push_to_hub("UrukHan/t5-russian-summarization", create_pr=1) 

CommitInfo(commit_url='https://huggingface.co/UrukHan/t5-russian-summarization/commit/5606c0f27943c33f9f088b182e1bf190b14eb952', commit_message='Upload tokenizer', commit_description='', oid='5606c0f27943c33f9f088b182e1bf190b14eb952', pr_url='https://huggingface.co/UrukHan/t5-russian-summarization/discussions/5', pr_revision='refs/pr/5', pr_num=5)

In [76]:
train = raw_datasets['train']
test = raw_datasets['test'].train_test_split(0.02)['test']  # Уменьшил так тестовыу. выборку чтоб не ждать долго расчет ошибок между эпохами

In [60]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #return_tensors="tf"

In [61]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  # Rouge expects a newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  # Extract a few results
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  
  # Add mean generated length
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

In [62]:
training_args = Seq2SeqTrainingArguments(
  output_dir = REPO,
  #overwrite_output_dir=True,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  eval_steps=5000,
  save_steps=5000,
  num_train_epochs=1,
  predict_with_generate=True,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  fp16=True,
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  #logging_dir='logs',
  push_to_hub=True,
)

In [63]:
# Выберем вручную оптимизатор. Т5 в оригинальной архитектуре использует Адафактор оптимизатор
optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

In [68]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = train,
  eval_dataset = test,
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics
)

/content/t5-russian-summarization is already a clone of https://huggingface.co/Katerina-gopher/t5-russian-summarization. Make sure you pull the latest changes with `repo.git_pull()`.


In [69]:
trainer.train()

IndexError: ignored